# Data Cleaning

In [1]:
DATA_PATH = "../output/raw-csv"
OUTPUT_PATH = "../output/csv"
OUTPUT_STAT_PATH = "../output/csv-stat"

OUTLIER_THRESHOLDS = {
    "pH": (0, 14),
    "EC": (0, 2000),
    "Temp": (0, 50),
    "DO": (0, 20)
}

In [ ]:
from pathlib import Path
from pandas import DataFrame, read_csv
from logging import basicConfig, INFO
from dataclasses import dataclass
from IPython.display import display

basicConfig(level=INFO)

from mp import mp_print, mp_exec

INFO:mp:Set start method to fork


In [3]:
DATA_PATH = Path(DATA_PATH).resolve()
OUTPUT_PATH = Path(OUTPUT_PATH).resolve()
OUTPUT_STAT_PATH = Path(OUTPUT_STAT_PATH).resolve()

OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
OUTPUT_STAT_PATH.mkdir(parents=True, exist_ok=True)

In [4]:
def iter_files():
    for file in filter(lambda x: x.is_file() and x.is_file(), DATA_PATH.iterdir()):
        yield file

In [5]:
@dataclass
class Stat:
    station: str
    total: int
    duplicate: int
    missing: int
    outliers: int
    valid: int
    threshold_outliers: int
    iqr_outliers: int
    ph_iqr: float
    ph_lb: float
    ph_ub: float
    ec_iqr: float
    ec_lb: float
    ec_ub: float
    temp_iqr: float
    temp_lb: float
    temp_ub: float
    do_iqr: float
    do_lb: float
    do_ub: float

def task(file: Path):
    station = file.name.split(".")[0]
    mp_print(f"Processing : {station}")
    
    stat = Stat(station,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
    df = read_csv(file, parse_dates=["Datetime"])
    stat.total = len(df)

    df = df[["Datetime", "pH", "EC", "Temp", "DO"]]

    size = len(df)
    df.dropna(inplace=True)
    stat.missing = size - len(df)

    size = len(df)
    df.drop_duplicates(inplace=True)
    df.drop_duplicates(subset=["pH", "EC", "Temp", "DO"], inplace=True)
    stat.duplicate = size - len(df)

    size = len(df)
    for column, (low, high) in OUTLIER_THRESHOLDS.items():
        df = df[(df[column] >= low) & (df[column] <= high)]
        stat.threshold_outliers += size - len(df)
        size = len(df)

    Q1 = df[["pH", "EC", "Temp", "DO"]].quantile(0.25)
    Q3 = df[["pH", "EC", "Temp", "DO"]].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df = df[
        (df["pH"] >= lower_bound["pH"]) & (df["pH"] <= upper_bound["pH"]) &
        (df["EC"] >= lower_bound["EC"]) & (df["EC"] <= upper_bound["EC"]) &
        (df["Temp"] >= lower_bound["Temp"]) & (df["Temp"] <= upper_bound["Temp"]) &
        (df["DO"] >= lower_bound["DO"]) & (df["DO"] <= upper_bound["DO"])
    ]
    
    for column in ["pH", "EC", "Temp", "DO"]:
        setattr(stat, f"{column.lower()}_iqr", IQR[column])
        setattr(stat, f"{column.lower()}_lb", lower_bound[column])
        setattr(stat, f"{column.lower()}_ub", upper_bound[column])

    stat.iqr_outliers = size - len(df)
    stat.outliers = stat.threshold_outliers + stat.iqr_outliers
    stat.valid = len(df)

    assert stat.total == stat.missing + stat.duplicate + stat.outliers + stat.valid

    return (station, df, stat)

In [6]:
data: dict[str, DataFrame] = {x: (y, z) for x, y, z in mp_exec(task, iter_files(), unorder=True)}

stats: list[Stat] = []

for station, values in data.items():
    df, stat = values
    print(f"Station : {station}, data : {df.shape}")
    stats.append(stat)
    df.to_csv(OUTPUT_PATH / f"{station}.csv", index=False)

Processing : ป่าสัก3สถานี แก่งคอย 2558-2563
Processing : แม่น้ำท่าจีน สองพี่น้อง 2558-2563
Processing : ป่าสัก3สถานี เสาไห้ 2558-2563
Processing : ป่าสัก3สถานี นครหลวง 2558-2563
Processing : ยม สามง่าม 2558-2563
Processing : ยม สุโขทัย 2558-2563
Processing : ยม โพทะเล 2558-2563
Processing : แม่น้ำท่าจีน กระทุ่มแบน 2558-2563
Processing : วัง เกาะคา 2558-2563
Processing : แม่น้ำท่าจีน บางเลน 2558-2563
Processing : แม่น้ำท่าจีน นครชัยศรี 2558-2563
Processing : แม่น้ำท่าจีน หันคา 2558-2563
Processing : แม่น้ำท่าจีน สุพรรณบุรี 2558-2563


/tmp/ipykernel_1554840/3908796547.py:29: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = read_csv(file, parse_dates=["Datetime"])


Processing : แม่น้ำท่าจีน สามชุก 2558-2563
Processing : ปิง เชียงใหม่ 2558-2563
Processing : ปิง กำแพงเพชร 2558-2563
Processing : น่าน อุตรดิตถ์ 2558-2563
Processing : น่าน น่าน 2558-2563
Processing : น่าน พิษณุโลก 2558-2563
Station : ยม สุโขทัย 2558-2563, data : (5736, 5)
Station : ยม สามง่าม 2558-2563, data : (49549, 5)
Station : วัง เกาะคา 2558-2563, data : (39386, 5)
Station : ยม โพทะเล 2558-2563, data : (30122, 5)
Station : แม่น้ำท่าจีน สองพี่น้อง 2558-2563, data : (52552, 5)
Station : แม่น้ำท่าจีน นครชัยศรี 2558-2563, data : (38656, 5)
Station : ป่าสัก3สถานี แก่งคอย 2558-2563, data : (45771, 5)
Station : แม่น้ำท่าจีน สามชุก 2558-2563, data : (6793, 5)
Station : แม่น้ำท่าจีน สุพรรณบุรี 2558-2563, data : (64990, 5)
Station : แม่น้ำท่าจีน หันคา 2558-2563, data : (41256, 5)
Station : แม่น้ำท่าจีน บางเลน 2558-2563, data : (63101, 5)
Station : ป่าสัก3สถานี เสาไห้ 2558-2563, data : (52813, 5)
Station : แม่น้ำท่าจีน กระทุ่มแบน 2558-2563, data : (28329, 5)
Station : ป่าสัก3สถานี นครหลวง 2

In [7]:
stats_df = DataFrame([s.__dict__ for s in stats])
stats_df.set_index("station", inplace=True)
stats_df.sort_values("valid", ascending=False, inplace=True)
stats_df.to_csv(OUTPUT_STAT_PATH / "stat.csv", index=True)

display(stats_df[["total", "missing", "duplicate", "outliers", "threshold_outliers", "iqr_outliers", "valid"]])

,total,missing,duplicate,outliers,threshold_outliers,iqr_outliers,valid
station,,,,,,,
แม่น้ำท่าจีน สุพรรณบุรี 2558-2563,77994,8473,1408,3123,0,3123,64990
ป่าสัก3สถานี นครหลวง 2558-2563,85184,10056,502,9650,0,9650,64976
แม่น้ำท่าจีน บางเลน 2558-2563,98746,20740,1020,13885,0,13885,63101
น่าน พิษณุโลก 2558-2563,88265,8125,6383,13399,0,13399,60358
ป่าสัก3สถานี เสาไห้ 2558-2563,70819,2343,5285,10378,1991,8387,52813
แม่น้ำท่าจีน สองพี่น้อง 2558-2563,74837,6647,3460,12178,161,12017,52552
น่าน อุตรดิตถ์ 2558-2563,96165,29075,1685,13231,1100,12131,52174
น่าน น่าน 2558-2563,94835,32221,1218,10073,0,10073,51323
ปิง กำแพงเพชร 2558-2563,82788,20834,2119,9067,2,9065,50768


In [8]:
display(stats_df[[f"{x}_{y}" for y in ["iqr", "lb", "ub"] for x in ["ph", "ec", "temp", "do"]]].round(3))

,ph_iqr,ec_iqr,temp_iqr,do_iqr,ph_lb,ec_lb,temp_lb,do_lb,ph_ub,ec_ub,temp_ub,do_ub
station,,,,,,,,,,,,
แม่น้ำท่าจีน สุพรรณบุรี 2558-2563,2.700,127.200,2.632,2.70,0.950,63.900,25.520,-1.75,11.750,572.700,36.048,9.05
ป่าสัก3สถานี นครหลวง 2558-2563,0.800,107.375,2.600,2.00,5.300,122.062,25.500,-0.20,8.500,551.562,35.900,7.80
แม่น้ำท่าจีน บางเลน 2558-2563,0.700,193.100,6.890,2.30,5.150,-70.050,14.376,-2.15,7.950,702.350,41.935,7.05
น่าน พิษณุโลก 2558-2563,1.300,37.000,2.600,1.70,3.950,99.100,22.700,2.55,9.150,247.100,33.100,9.35
ป่าสัก3สถานี เสาไห้ 2558-2563,1.290,129.700,2.500,2.78,5.865,152.550,25.450,-0.35,11.025,671.350,35.450,10.77
แม่น้ำท่าจีน สองพี่น้อง 2558-2563,1.080,127.900,2.100,1.10,4.180,166.850,26.350,-0.25,8.500,678.450,34.750,4.15
น่าน อุตรดิตถ์ 2558-2563,2.148,89.500,3.600,1.30,1.930,45.150,20.600,3.55,10.522,403.150,35.000,8.75
น่าน น่าน 2558-2563,0.700,59.525,4.200,1.10,6.150,113.312,20.400,4.85,8.950,351.412,37.200,9.25
ปิง กำแพงเพชร 2558-2563,1.100,40.900,2.400,0.90,4.850,161.750,24.600,4.55,9.250,325.350,34.200,8.15
